In [20]:
import numpy as np
import random

# Given parameters
states = 10
actions = [-1, 1]  # -1: move left, 1: move right
prob_stay = 0.5
prob_move = 0.5
discount_factor = 0.9  # Discount factor
g = np.array([1, 2, 3, 4, 5, 4, 2, 0, 1, 2])  # Cost array

# Transition probabilities
P = np.zeros((states + 1, states + 1, len(actions)))

for k in range(1, states + 1):
    for u_index, u in enumerate(actions):
        if u == -1:  # Moving left
            for j in range(1, states + 1):
                if j == k:
                    P[k, j, u_index] = prob_stay
                elif j == k - 1:
                    P[k, j, u_index] = prob_move
        elif u == 1:  # Moving right
            for j in range(1, states + 1):
                if j == k:
                    P[k, j, u_index] = prob_stay
                elif j == k + 1:
                    P[k, j, u_index] = prob_move

# Policy iteration
policy = np.zeros(states + 1, dtype=int)  # Initialize policy randomly

def evaluate_policy(policy, V):
    while True:
        V_old = np.copy(V)
        for k in range(1, states + 1):
            u = policy[k]
            expected_value = 0
            for u_index, action in enumerate(actions):
                next_state = min(max(k + action, 1), states)  # Ensure next state is within bounds
                expected_value += P[k, next_state, u_index] * (g[k - 1] + discount_factor * V[next_state])
            V[k] = expected_value
        if np.max(np.abs(V - V_old)) < 1e-6:
            break

def improve_policy(policy, V):
    policy_stable = True
    for k in range(1, states + 1):
        old_action = policy[k]
        action_values = np.zeros(len(actions))
        for u_index, u in enumerate(actions):
            next_state = min(max(k + u, 1), states)  # Ensure next state is within bounds
            action_values[u_index] = g[k - 1] + discount_factor * V[next_state]
        best_actions = [action for action, val in enumerate(action_values) if val == min(action_values)]
        new_action = random.choice(best_actions)
        policy[k] = actions[new_action]
        if old_action != actions[new_action]:
            policy_stable = False
    return policy_stable

V = np.zeros(states + 1)  # Initialize value function

policy_stable = False
while not policy_stable:
    evaluate_policy(policy, V)
    policy_stable = improve_policy(policy, V)

# Print results
print("Optimal Policy:")
print(policy[1:])  # Exclude absorbing state
print("\nOptimal Value Function:")
print(V[1:])  # Exclude absorbing state


Optimal Policy:
[-1 -1 -1 -1  1  1  1  1  1 -1]

Optimal Value Function:
[21.62645012 24.21010711 27.729344   30.74399114 31.70174785 28.59322669
 22.94986738 17.96203449 16.96576515 17.51744394]
